# **ЭТОТ НАУТБУК ЛУЧШЕ ВСЕГО ОПИСЫВАЕТ СЛОВО ПСЕВДОКОД БУДЕТ НЕВЕРОЯТНОЕ КОЛИЧЕСТВО БАГОВ, БЕЗ ТРЕЙНА НЕВОЗМОЖНО ЗАПРЕДИКТИТЬ**

In [ ]:
!pip install -U accelerate
!pip install transformers==4.45.2 sentence-transformers==3.1.1

In [6]:
import pandas as pd
import torch
import numpy as np
import random
import os
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, SentenceTransformerTrainingArguments, SentencesDataset, losses, CrossEncoder, InputExample, evaluation
from sklearn.metrics.pairwise import cosine_similarity


In [7]:
seed = 42

random.seed(seed)
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [ ]:
df = pd.read_csv(path)
test = pd.read_csv(path)

df = pd.DataFrame({
    'text1': ["текст1", "текст2", "текст3"],
    'text2': ["текст4", "текст5", "текст6"],
    'text3': ["текст7", "текст8", "текст9"],
    'text4': ["текст10", "текст11", "текст12"],
    'text5': ["текст13", "текст14", "текст15"],
    'label': [0, 1, 0]
})

submission_example = pd.read_csv("submission_example.csv")

In [ ]:
pd.Variables_Predicts_Train = pd.DataFrame()
pd.Variables_Predicts_Test = pd.DataFrame()

# **CrossEncoder**

In [ ]:
Train, Val = train_test_split(df, test_size=0.2, random_state=42)

train = [
    InputExample(
        texts=[f"{row['context']}: {row['anchor']}", row['target']], 
        label=row['score']
    ) 
    for _, row in Train.iterrows()
]

val = [
    InputExample(
        texts=[f"{row['context']}: {row['anchor']}", row['target']], 
        label=row['score']
    ) 
    for _, row in Val.iterrows()
]

train_dataloader = DataLoader(train, shuffle=True, batch_size=64)
evaluator = CECorrelationEvaluator.from_input_examples(val, name="val-eval")

model = CrossEncoder('deepvk/USER-bge-m3', num_labels=1, device='cuda')

model.fit(
    train_dataloader=train_dataloader,
    evaluator=evaluator,  
    epochs=1,
    warmup_steps=100,
    evaluation_steps=100,
    callback=validation_callback,
)


Test = pd.DataFrame({
    'context': ["контекст1", "контекст2", "контекст3"],
    'anchor': ["якорь1", "якорь2", "якорь3"],
    'target': ["цель1", "цель2", "цель3"]
})


test_texts = [[f"{row['context']}: {row['anchor']}", row['target']] for _, row in Test.iterrows()]

predict = model.predict(test_texts)
submission_example['label'] = predict
submission_example.to_csv("submission.csv", index=False)
Variables_Predicts_Test['Baseline_pred'] = predict

train_texts = [
    [f"{row['context']}: {row['anchor']}", row['target']] for _, row in Train.iterrows()
]
predictions = model.predict(train_texts)

Variables_Predicts_Train['Baseline_pred'] = predict

# **Base cos sim**

In [ ]:
model = SentenceTransformer("deepvk/USER-bge-m3")

def take_sim(df):
    history = df[col for col in df.columns if col != ['label' , 'query']]
    average_embeddings = np.array([
    np.mean(model.encode(row[text_columns].tolist()), axis=0) for _, row in history.iterrows()
    ])
    query = df['query']
    query_embeddings = model.encode(query.tolist())

    similarities = cosine_similarity(average_embeddings, query_embeddings)
    
    return similarities
    
predict = take_sim(test)
Variables_Predicts_Test['Baseline_pred_1'] = predict
submission_example['label'] = predict
submission_example.to_csv("submission.csv", index=False)


predict = take_sim(df)
Variables_Predicts_Train['Baseline_pred_1'] = predict

# **Trainig Encoder**

In [ ]:
Train, Val = train_test_split(train, test_size=0.2, random_state=42)

train_dataset = Dataset.from_dict({
    "sentence1": [row['combined'] for _, row in train.iterrows()],
    "sentence2": [row['target'] for _, row in train.iterrows()],
    "score": [row['score'] for _, row in train.iterrows()],
})

val_dataset = Dataset.from_dict({
    "sentence1": [row['combined'] for _, row in Val.iterrows()],
    "sentence2": [row['target'] for _, row in Val.iterrows()],
    "score": [row['score'] for _, row in Val.iterrows()],
})

evaluator = evaluation.EmbeddingSimilarityEvaluator(
    val_dataset["sentence1"],
    val_dataset["sentence2"],
    val_dataset["score"]
)

training_args = SentenceTransformerTrainingArguments(
    output_dir="/home/pret/PycharmProjects/Vseros_classification/Models/mpnet-base",
    num_train_epochs=3,
    eval_strategy="steps",
    eval_steps=500,
    logging_dir="/home/pret/PycharmProjects/Vseros_classification/Models/",
    logging_steps=100,
    save_steps=1000,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=100,
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True
)

# Инициализация модели и loss
model = SentenceTransformer("all-mpnet-base-v2")
loss = losses.CoSENTLoss(model)

# Обучение модели с Trainer
trainer = SentenceTransformerTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    evaluator=evaluator,
 #   compute_metrics=compute_metrics,
    loss=loss,
 #   callbacks=[CustomCallback()],
    args=training_args
)

trainer.train()

def take_sim(df):
    history = df[col for col in df.columns if col != ['label' , 'query']]
    average_embeddings = np.array([
    np.mean(model.encode(row[text_columns].tolist()), axis=0) for _, row in history.iterrows()
    ])
    query = df['query']
    query_embeddings = model.encode(query.tolist())

    similarities = cosine_similarity(average_embeddings, query_embeddings)
    
    return similarities
    
predict = take_sim(test)
Variables_Predicts_Test['Fine_tuning_pred'] = predict
submission_example['label'] = predict
submission_example.to_csv("submission.csv", index=False)


predict = take_sim(df)
Variables_Predicts_Train['Fine_tuning_pred'] = predict

In [ ]:
def transform_text_columns(df, model):
    for col in df.columns:
        if col != 'lable'
        embeddings = model.encode(df[col].tolist())
        df[col] = list(np.array(embeddings))
    return df

df = transform_text_columns(df)
test = transform_text_columns(test)

# **Catboost on embeddings**

In [ ]:
for_cat_data = pd.DataFrame()

embedding_cols = [col for col in df.columns if not in ['query', 'target']]
def compute_average_embedding(row, embedding_cols):
    embeddings = np.array([row[col] for col in embedding_cols])
    average = embeddings.mean(axis=0)
    return average

df['average_embedding'] = df.apply(lambda row: compute_average_embedding(row), axis=1)
test['average_embedding'] = test.apply(lambda row: compute_average_embedding(row), axis=1)

def expand_embeddings(df, embedding_cols):
    for embedding_col in embedding_cols:
        # Создаем новые колонки для каждого элемента эмбеддинга
        embedding_size = len(df[embedding_col][0])  # размер эмбеддинга
        expanded_cols = {f"{embedding_col}_{i}": df[embedding_col].apply(lambda x: x[i]) for i in range(embedding_size)}
        
        # Добавляем новые колонки в DataFrame
        df = pd.concat([df, pd.DataFrame(expanded_cols)], axis=1)
        
        # Удаляем исходный столбец с эмбеддингом
        df = df.drop(columns=[embedding_col])
    
    return df

# Применяем функцию для раскладывания эмбеддингов
for_cat_data = expand_embeddings(df, embedding_cols=['average_embedding', 'query'])
for_cat_data['target'] = df['target']

test_cat_data = expand_embeddings(df, embedding_cols=['average_embedding', 'query'])

In [ ]:
train_df, val_df = train_test_split(df, test_size=.20, random_state=2)

train_df.reset_index(drop=True, 
               inplace=True)
val_df.reset_index(drop=True, 
                 inplace=True)

y = train_df['target']
X = train_df.drop(columns = 'target')
y_val = val_df['target']
X_val = val_df.drop(columns = 'target')


model = CatBoostClassifier(
    #scale_pos_weight=scale_pos_weight_value,
    auto_class_weights='SqrtBalanced',
    #auto_class_weights='Balanced',
    loss_function='Logloss',
    eval_metric='AUC',
    random_seed=42,
    logging_level='Silent',
    iterations=1000,
    task_type="GPU",
    devices='0',
    use_best_model=True,
    early_stopping_rounds=100
)


model.fit(
    X, y,
    cat_features=cats,
    eval_set=[(X_val, y_val)],
    logging_level='Verbose', #Verbose
)
    
print(" AUC по test: ", model.get_best_score()['validation']['AUC'])

predict = model.predict_proba(test_cat_data)[:,1:].flatten()
submission_example['label'] = predict
submission_example.to_csv("submission.csv", index=False)
Variables_Predicts_Test['Catboost_emb'] = predict

predict = model.predict(for_cat_data)
Variables_Predicts_Train['Catboost_emb'] = predict

# **Catboost on values**

In [ ]:
for_cat_data = pd.DataFrame()
embedding_cols = [col for col in df.columns if col not in ['query', 'target']]
query_column = 'query'

# Функция для вычисления косинусного сходства
def calculate_cosine_similarity(row, query_col, embedding_col):
    return cosine_similarity([row[query_col]], [row[embedding_col]])[0, 0]

# Заменяем эмбеддинги на косинусное сходство с `query`
for col in embedding_cols:
    for_cat_data[col] = df.apply(lambda row: calculate_cosine_similarity(row, query_column, col), axis=1)
    test_cat_data[col] = df.apply(lambda row: calculate_cosine_similarity(row, query_column, col), axis=1)

In [ ]:
train_df, val_df = train_test_split(df, test_size=.20, random_state=2)

train_df.reset_index(drop=True, 
               inplace=True)
val_df.reset_index(drop=True, 
                 inplace=True)

y = train_df['target']
X = train_df.drop(columns = 'target')
y_val = val_df['target']
X_val = val_df.drop(columns = 'target')


model = CatBoostClassifier(
    #scale_pos_weight=scale_pos_weight_value,
    auto_class_weights='SqrtBalanced',
    #auto_class_weights='Balanced',
    loss_function='Logloss',
    eval_metric='AUC',
    random_seed=42,
    logging_level='Silent',
    iterations=1000,
    task_type="GPU",
    devices='0',
    use_best_model=True,
    early_stopping_rounds=100
)

model.fit(
    X, y,
    cat_features=cats,
    eval_set=[(X_val, y_val)],
    logging_level='Verbose', #Verbose
)
    
print(" AUC по test: ", model.get_best_score()['validation']['AUC'])

predict = model.predict_proba(test_cat_data)[:,1:].flatten()
submission_example['label'] = predict
submission_example.to_csv("submission.csv", index=False)
Variables_Predicts_Test['Catboost_emb'] = predict

predict = model.predict(for_cat_data)
Variables_Predicts_Train['Catboost_emb'] = predict

# **Bert embmean**

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torch.nn.functional import cosine_similarity
import pandas as pd
from tqdm import tqdm

# Устройство для работы с моделью
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Класс для модели с двумя потоками для query и positive/negative
class EmbeddingModel(nn.Module):
    def __init__(self, model_name="USER-bge-m3"):
        super().__init__()
        self.device = device
        self.embedding_model = AutoModel.from_pretrained(model_name).to(self.device)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

    def forward(self, input_texts):
        encoding = self.tokenizer.batch_encode_plus(
            input_texts,
            padding="max_length",
            max_length=128,
            truncation=True,
            return_tensors='pt',
            add_special_tokens=True
        )
        input_ids = encoding['input_ids'].to(self.device)
        attention_mask = encoding['attention_mask'].to(self.device)

        # Извлекаем эмбеддинги текста
        outputs = self.embedding_model(input_ids=input_ids, attention_mask=attention_mask)
        embeddings = outputs.last_hidden_state.mean(dim=1)  # Усредняем по длине последовательности
        return embeddings

# Dataset для DataLoader
class ContrastiveTextDataset(Dataset):
    def __init__(self, queries, positives, labels):
        self.queries = queries
        self.positives = positives
        self.labels = labels

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return [self.queries[idx], self.positives[idx], self.labels[idx]]

# Класс для обучения и валидации с косинусным расстоянием
class ContrastiveTrainer:
    def __init__(self, model):
        self.model = model

    def train(self, dataloader, optimizer, loss_func, epochs, val_dataloader):
        self.model.train()
        for epoch in range(epochs):
            for queries, positives, labels in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}", colour="GREEN"):
                labels = labels.to(device).float()
                optimizer.zero_grad()

                # Получение эмбеддингов для query и positive/negative
                query_embeddings = self.model(queries)
                positive_embeddings = self.model(positives)

                # Расчет косинусной близости
                cos_sim = cosine_similarity(query_embeddings, positive_embeddings)
                loss = loss_func(cos_sim, labels)  # Контрастивная функция потерь
                loss.backward()
                optimizer.step()
                scheduler.step()

            self.evaluate(val_dataloader, loss_func)

    def evaluate(self, dataloader, loss_func):
        self.model.eval()
        test_loss, correct = 0, 0
        size = len(dataloader.dataset)
        num_batches = len(dataloader)
        
        with torch.no_grad():
            for queries, positives, labels in tqdm(dataloader, desc="Evaluation", colour="CYAN"):
                labels = labels.to(device).float()
                query_embeddings = self.model(queries)
                positive_embeddings = self.model(positives)
                cos_sim = cosine_similarity(query_embeddings, positive_embeddings)
                loss = loss_func(cos_sim, labels)
                test_loss += loss.item()
                preds = (cos_sim > 0.5).float()  # Прогнозы на основе косинусной близости
                correct += (preds == labels).type(torch.float).sum().item()

        accuracy = correct / size
        avg_loss = test_loss / num_batches
        print(f"Validation: \nAccuracy: {accuracy*100:.1f}%, Avg loss: {avg_loss:.4f} \n")

    def create_result(self, test_data):
        self.model.eval()
        queries = test_data['query'].values.tolist()
        positives = test_data['positive'].values.tolist()
        
        results = []
        for query, positive in tqdm(zip(queries, positives), desc="Generating Results", colour="CYAN"):
            query_emb = self.model([query])
            positive_emb = self.model([positive])
            cos_sim = cosine_similarity(query_emb, positive_emb)
            results.append(cos_sim.item())

        submission = pd.DataFrame({'id': test_data['id'], 'similarity': results})
        return submission

    def predict_proba(self, queries, positives):
        self.model.eval()
        query_emb = self.model(queries)
        positive_emb = self.model(positives)
        cos_sim = cosine_similarity(query_emb, positive_emb)
        return cos_sim

# Сплит данных
queries = train_data['query'].values.tolist()
positives = train_data['positive'].values.tolist()
labels = train_data['label'].values.tolist()
X_train_q, X_val_q, X_train_p, X_val_p, y_train, y_val = train_test_split(
    queries, positives, labels, test_size=0.2)

# Создание датасетов и DataLoader
train_dataset = ContrastiveTextDataset(X_train_q, X_train_p, y_train)
val_dataset = ContrastiveTextDataset(X_val_q, X_val_p, y_val)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=4)

# Инициализация модели и компонентов обучения
model = EmbeddingModel()
epochs = 10
learning_rate = 1e-6
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-4, steps_per_epoch=len(train_loader), epochs=epochs)
loss_func = nn.CosineEmbeddingLoss()

# Обучение и валидация
trainer = ContrastiveTrainer(model)
trainer.train(train_loader, optimizer, loss_func, epochs, val_loader)

# Создание предсказаний для теста
predict_test = trainer.predict_proba(test_data['query'].values.tolist(), test_data['positive'].values.tolist())
test_data['label'] = predict_test.flatten()

# Сохранение предсказаний в CSV для подачи в соревнование
test_data.to_csv("submission.csv", index=False)

# Ансамблирование (например, добавление прогнозов других моделей)
Variables_Predicts_Test['Model_emb'] = predict_test.flatten()
Variables_Predicts_Train['Model_emb'] = trainer.predict_proba(train_data['query'].values.tolist(), train_data['positive'].values.tolist()).flatten()

# **Query Bert**

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm
from torch.nn.functional import cosine_similarity

# Устройство для работы с моделью
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Класс модели с расчетом эмбеддингов текста и query
class EmbeddingSimilarityModel(nn.Module):
    def __init__(self, model_name="USER-bge-m3"):
        super().__init__()
        self.device = device
        self.embedding_model = AutoModel.from_pretrained(model_name).to(self.device)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

    def forward(self, text_inputs, query_inputs):
        # Кодируем текстовые эмбеддинги
        text_encoding = self.tokenizer.batch_encode_plus(
            text_inputs,
            padding="max_length",
            max_length=128,
            truncation=True,
            return_tensors='pt',
            add_special_tokens=True
        )
        query_encoding = self.tokenizer.batch_encode_plus(
            query_inputs,
            padding="max_length",
            max_length=128,
            truncation=True,
            return_tensors='pt',
            add_special_tokens=True
        )

        # Получаем эмбеддинги для текста и запроса
        text_embeds = self.embedding_model(
            input_ids=text_encoding['input_ids'].to(self.device),
            attention_mask=text_encoding['attention_mask'].to(self.device)
        ).last_hidden_state.mean(dim=1)

        query_embeds = self.embedding_model(
            input_ids=query_encoding['input_ids'].to(self.device),
            attention_mask=query_encoding['attention_mask'].to(self.device)
        ).last_hidden_state.mean(dim=1)

        # Считаем косинусное расстояние
        cos_similarity = cosine_similarity(text_embeds, query_embeds, dim=-1)
        return cos_similarity

# Dataset для DataLoader
class TextPairDataset(Dataset):
    def __init__(self, texts, queries, labels):
        self.texts = texts
        self.queries = queries
        self.labels = labels

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return [self.texts[idx], self.queries[idx], self.labels[idx]]

# Класс для обучения и валидации
class ModelTrainer:
    def __init__(self, model):
        self.model = model

    def train(self, dataloader, optimizer, loss_func, epochs, val_dataloader):
        self.model.train()
        for epoch in range(epochs):
            for texts, queries, labels in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}", colour="GREEN"):
                labels = labels.to(device).float()  # Для CosineEmbeddingLoss метки -1 или 1
                optimizer.zero_grad()
                cos_sim = self.model(texts, queries)
                loss = loss_func(cos_sim, labels)  # Потеря на косинусном расстоянии
                loss.backward()
                optimizer.step()
                scheduler.step()
            
            self.evaluate(val_dataloader, loss_func)

            # Сохранение модели на каждой эпохе
            checkpoint = {'model': self.model, 'state_dict': self.model.state_dict()}
            torch.save(checkpoint, f'checkpoint_epoch_{epoch+1}.pth')

    def evaluate(self, dataloader, loss_func):
        self.model.eval()
        test_loss, correct = 0, 0
        size = len(dataloader.dataset)
        num_batches = len(dataloader)
        
        with torch.no_grad():
            for texts, queries, labels in tqdm(dataloader, desc="Evaluation", colour="CYAN"):
                labels = labels.to(device).float()
                cos_sim = self.model(texts, queries)
                loss = loss_func(cos_sim, labels)
                test_loss += loss.item()

        avg_loss = test_loss / num_batches
        print(f"Validation: Avg loss: {avg_loss:.4f} \n")

    def create_result(self, test_data, query_data):
        text_inputs = test_data['text'].values.tolist()
        query_inputs = query_data['query'].values.tolist()
        similarities = []

        with torch.no_grad():
            for text, query in zip(text_inputs, query_inputs):
                cos_sim = self.model([text], [query])
                similarities.append(cos_sim.item())

        submission = pd.DataFrame({'id': test_data['id'], 'similarity_score': similarities})
        return submission

    def predict_proba(self, queries, positives):
        self.model.eval()
        cos_sim = self.model(positives.values.tolist(), queries.values.tolist())
        return cos_sim

# Сплит данных
X_train, X_val, y_train, y_val = train_test_split(train_data[['text', 'query']].values.tolist(), train_data.target.values.tolist(), test_size=0.2)

# Создание датасетов и DataLoader
train_dataset = TextPairDataset([x[0] for x in X_train], [x[1] for x in X_train], y_train)
val_dataset = TextPairDataset([x[0] for x in X_val], [x[1] for x in X_val], y_val)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=4)

# Инициализация модели и компонентов обучения
model = EmbeddingSimilarityModel()
epochs = 10
learning_rate = 1e-6
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-4, steps_per_epoch=len(train_loader), epochs=epochs)
loss_func = nn.BCEWithLogitsLoss()

# Обучение и валидация
trainer = ModelTrainer(model)
trainer.train(train_loader, optimizer, loss_func, epochs, val_loader)

predict_test = trainer.predict_proba(test_data['positive'], test_data['query']).flatten()
test_data['label'] = predict_test

# Сохранение предсказаний в CSV для подачи в соревнование
test_data.to_csv("submission.csv", index=False)

# Ансамблирование (например, добавление прогнозов других моделей)
Variables_Predicts_Test['Model_query_emb'] = predict_test
Variables_Predicts_Train['Model_query_emb'] = trainer.predict_proba(train_data['positive'], train_data['query']).flatten()


# **Bertcat**

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm
from torch.nn.functional import cosine_similarity

# Устройство для работы с моделью
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Класс модели с расчетом эмбеддингов текста и query
class EmbeddingConcatModel(nn.Module):
    def __init__(self, model_name="USER-bge-m3"):
        super().__init__()
        self.device = device
        self.embedding_model = AutoModel.from_pretrained(model_name).to(self.device)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        
        # Линейная голова из трех слоев после конкатенации эмбеддингов
        self.classification_head = nn.Sequential(
            nn.Linear(2 * self.embedding_model.config.hidden_size, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        ).to(device)

    def forward(self, text_inputs, query_inputs):
        # Кодируем текстовые эмбеддинги
        text_encoding = self.tokenizer.batch_encode_plus(
            text_inputs, padding="max_length", max_length=128, truncation=True, return_tensors='pt', add_special_tokens=True
        )
        query_encoding = self.tokenizer.batch_encode_plus(
            query_inputs, padding="max_length", max_length=128, truncation=True, return_tensors='pt', add_special_tokens=True
        )

        # Получаем эмбеддинги для текста и запроса
        text_embeds = self.embedding_model(
            input_ids=text_encoding['input_ids'].to(self.device),
            attention_mask=text_encoding['attention_mask'].to(self.device)
        ).last_hidden_state.mean(dim=1)

        query_embeds = self.embedding_model(
            input_ids=query_encoding['input_ids'].to(self.device),
            attention_mask=query_encoding['attention_mask'].to(self.device)
        ).last_hidden_state.mean(dim=1)

        # Конкатенируем эмбеддинги
        combined_embeddings = torch.cat((text_embeds, query_embeds), dim=1)
        logits = self.classification_head(combined_embeddings)

        return logits

# Dataset для DataLoader
class TextPairDataset(Dataset):
    def __init__(self, texts, queries, labels):
        self.texts = texts
        self.queries = queries
        self.labels = labels

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return [self.texts[idx], self.queries[idx], self.labels[idx]]

# Класс для обучения и валидации
class ModelTrainer:
    def __init__(self, model):
        self.model = model

    def train(self, dataloader, optimizer, loss_func, epochs, val_dataloader):
        self.model.train()
        for epoch in range(epochs):
            for texts, queries, labels in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}", colour="GREEN"):
                labels = labels.to(device).float()
                optimizer.zero_grad()
                logits = self.model(texts, queries).squeeze()
                loss = loss_func(logits, labels)
                loss.backward()
                optimizer.step()
                scheduler.step()
            
            self.evaluate(val_dataloader, loss_func)

            # Сохранение модели на каждой эпохе
            checkpoint = {'model': self.model, 'state_dict': self.model.state_dict()}
            torch.save(checkpoint, f'checkpoint_epoch_{epoch+1}.pth')

    def evaluate(self, dataloader, loss_func):
        self.model.eval()
        test_loss = 0
        num_batches = len(dataloader)
        
        with torch.no_grad():
            for texts, queries, labels in tqdm(dataloader, desc="Evaluation", colour="CYAN"):
                labels = labels.to(device).float()
                logits = self.model(texts, queries).squeeze()
                loss = loss_func(logits, labels)
                test_loss += loss.item()

        avg_loss = test_loss / num_batches
        print(f"Validation: Avg loss: {avg_loss:.4f} \n")

    def predict_proba(self, dataloader):
        self.model.eval()
        all_preds = []
        
        with torch.no_grad():
            for texts, queries in tqdm(dataloader, desc="Predicting", colour="CYAN"):
                logits = torch.sigmoid(self.model(texts, queries)).squeeze().cpu().numpy()
                all_preds.extend(logits)
        
        return all_preds

# Сплит данных
X_train, X_val, y_train, y_val = train_test_split(
    train_data[['text', 'query']].values.tolist(), train_data['target'].values.tolist(), test_size=0.2
)

# Создание датасетов и DataLoader
train_dataset = TextPairDataset([x[0] for x in X_train], [x[1] for x in X_train], y_train)
val_dataset = TextPairDataset([x[0] for x in X_val], [x[1] for x in X_val], y_val)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=4)

# Инициализация модели и компонентов обучения
model = EmbeddingConcatModel()
epochs = 10
learning_rate = 1e-6
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-4, steps_per_epoch=len(train_loader), epochs=epochs)
loss_func = nn.BCEWithLogitsLoss()

# Обучение и валидация
trainer = ModelTrainer(model)
trainer.train(train_loader, optimizer, loss_func, epochs, val_loader)

# Предсказания на тестовой выборке
test_dataset = TextPairDataset(test_data['text'].values.tolist(), test_data['query'].values.tolist(), [0] * len(test_data))
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)

predict_test = trainer.predict_proba(test_loader)
test_data['label'] = predict_test

# Сохранение предсказаний в CSV для подачи в соревнование
test_data.to_csv("submission.csv", index=False)

# Ансамблирование
Variables_Predicts_Test['Model_cat_emb'] = predict_test
Variables_Predicts_Train['Model_cat_emb'] = trainer.predict_proba(train_loader)